# incorporate a clear sky model via pvlib


- The clear-sky model estimates the irradiance under ideal atmospheric conditions and can serve as a baseline for comparison with the actual measurements.

# Hybrid Approach:

* Combine the predictions from the machine learning model with the clear-sky model's predictions.
* Use a weighted average, ensemble, or other combination methods to blend the predictions.
* This hybrid approach can take advantage of both data-driven patterns and physical insights.


we use 2014 and 2015 as the training set and 2016 as the testing, remove night values according to the solar zenith angle (night :1⁄4 hz > 85), and select model hyperparameters using tenfold CrossValidation (CV). 

standard forecast error metrics: mean absolute error (MAE), mean bias error (MBE), root mean square error (RMSE), and forecast skill, which we compute using RMSE as defined by: 

R. Marquez, C. F. M. Coimbra, Proposed metric for evaluation of solar forecasting models, Journal of Solar Energy Engineering 135 (1) (2012) 011016–011016–9. doi:10.1115/1.4007496.

```
# forecast skill [-]:
#
#       s = 1 - RMSE_f / RMSE_p
#
# where RMSE_f and RMSE_p are the RMSE of the forecast model
# and reference baseline model, respectively.
rmse_p = np.sqrt(
    np.mean((group["{}_{}".format(target, horizon)] - group["{}_{}".format(target, baseline)]) ** 2)
)
skill = 1.0 - rmse / rmse_p

results
```

The addition of the exogenous features has a clear positive effect on forecast performance, with the significant improvement of the forecast skill. 

# k_{t} = clear sky index

In [1]:
import pandas as pd

pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_irradiance.csv', parse_dates=True, index_col='timeStamp')

,ghi,dni,dhi
timeStamp,,,
2014-01-02 08:00:00,0.0,0.0,0.0
2014-01-02 08:01:00,0.0,0.0,0.0
2014-01-02 08:02:00,0.0,0.0,0.0
2014-01-02 08:03:00,0.0,0.0,0.0
2014-01-02 08:04:00,0.0,0.0,0.0
...,...,...,...
2016-12-31 07:55:00,0.0,0.0,0.0
2016-12-31 07:56:00,0.0,0.0,0.0
2016-12-31 07:57:00,0.0,0.0,0.0


In [2]:
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.579454_lon-121.260320.csv', parse_dates=True, skiprows=1, index_col='reftime')

,valtime,dwsw,cloud_cover,precipitation,pressure,wind-u,wind-v,temperature,rel_humidity
reftime,,,,,,,,,
2014-01-01 12:00:00,2014-01-02 14:00:00,0.000,2.0,0.000,101560.0,-1.2019,-0.8152,273.254,53.0
2014-01-01 12:00:00,2014-01-02 15:00:00,0.000,0.0,0.000,101546.0,-1.0587,-0.6262,272.865,53.0
2014-01-01 12:00:00,2014-01-02 16:00:00,95.000,0.0,0.000,101576.0,-0.8595,-0.3518,276.175,52.0
2014-01-01 12:00:00,2014-01-02 17:00:00,270.875,0.0,0.000,101649.0,-0.4019,-0.0743,283.253,46.0
2014-01-01 12:00:00,2014-01-02 18:00:00,420.750,16.0,0.000,101657.0,0.4367,-0.5003,288.350,40.0
...,...,...,...,...,...,...,...,...,...
2016-12-31 12:00:00,2017-01-01 23:00:00,98.250,100.0,0.375,100790.0,1.0159,-0.2391,281.583,81.0
2016-12-31 12:00:00,2017-01-02 00:00:00,40.125,100.0,0.375,100810.0,1.1071,0.3261,280.958,81.0
2016-12-31 12:00:00,2017-01-02 03:00:00,0.000,100.0,0.000,100855.0,0.0090,1.8306,279.198,90.0


In [3]:
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.579454_lon-121.260320.csv', parse_dates=True, skiprows=1, index_col='reftime')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.599891_lon-121.126680.csv', parse_dates=True, skiprows=1, index_col='reftime')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.683880_lon-121.286556.csv', parse_dates=True, skiprows=1, index_col='reftime')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.704328_lon-121.152788.csv', parse_dates=True, skiprows=1, index_col='reftime')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_irradiance.csv', parse_dates=True,index_col='timeStamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_satellite.csv', parse_dates=True)
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_sky_image_features.csv', parse_dates=True,index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_weather.csv', parse_dates=True, index_col='timeStamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Irradiance_features_day-ahead.csv', parse_dates=True,index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Irradiance_features_intra-day.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/NAM_nearest_node_day-ahead.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Sat_image_features_intra-day.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Irradiance_features_intra-hour.csv', parse_dates=True,  index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Target_intra-hour.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Sky_image_features_intra-hour.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Target_intra-day.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Target_day-ahead.csv', parse_dates=True, index_col='timestamp')


,ghi_26h,dni_26h,ghi_clear_26h,dni_clear_26h,ghi_kt_26h,dni_kt_26h,elevation_26h,ghi_27h,dni_27h,ghi_clear_27h,...,ghi_kt_39h,dni_kt_39h,elevation_39h,ghi_40h,dni_40h,ghi_clear_40h,dni_clear_40h,ghi_kt_40h,dni_kt_40h,elevation_40h
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-01-02 12:00:00,0.0,0.0,0.0,0.0,1.2,1.2,-21.277969,0.0,0.0,0.0,...,1.2,1.2,-17.944421,0.0,0.0,0.0,0.0,1.2,1.2,-29.419632
2014-01-03 12:00:00,0.0,0.0,0.0,0.0,1.2,1.2,-21.306546,0.0,0.0,0.0,...,1.2,1.2,-17.797112,0.0,0.0,0.0,0.0,1.2,1.2,-29.273474
2014-01-04 12:00:00,0.0,0.0,0.0,0.0,1.2,1.2,-21.329561,0.0,0.0,0.0,...,1.2,1.2,-17.646804,0.0,0.0,0.0,0.0,1.2,1.2,-29.124605
2014-01-05 12:00:00,0.0,0.0,0.0,0.0,1.2,1.2,-21.346937,0.0,0.0,0.0,...,1.2,1.2,-17.493609,0.0,0.0,0.0,0.0,1.2,1.2,-28.973134
2014-01-06 12:00:00,0.0,0.0,0.0,0.0,1.2,1.2,-21.358603,0.0,0.0,0.0,...,1.2,1.2,-17.337635,0.0,0.0,0.0,0.0,1.2,1.2,-28.819164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-26 12:00:00,0.0,0.0,0.0,0.0,1.2,1.2,-20.948323,0.0,0.0,0.0,...,1.2,1.2,-18.847297,0.0,0.0,0.0,0.0,1.2,1.2,-30.322399
2016-12-27 12:00:00,0.0,0.0,0.0,0.0,1.2,1.2,-21.012061,0.0,0.0,0.0,...,1.2,1.2,-18.723261,0.0,0.0,0.0,0.0,1.2,1.2,-30.197535
2016-12-28 12:00:00,0.0,0.0,0.0,0.0,1.2,1.2,-21.070834,0.0,0.0,0.0,...,1.2,1.2,-18.595457,0.0,0.0,0.0,0.0,1.2,1.2,-30.069196


Exogenous variables are independent, and endogenous variables are dependent. Therefore, if the variable does not depend on variables within the model, it's an exogenous variable. If the variable depends on variables within the model, though, it's endogenous.

# train / test split

In [4]:
from root import ROOT_DIR
ROOT_DIR.as_posix()

'/Users/adam/Documents/Projects/irradianceforecasting'

In [5]:
inpEndo = pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Irradiance_features_intra-hour.csv', parse_dates=True,  index_col='timestamp')
inpExo  = pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Sky_image_features_intra-hour.csv', parse_dates=True, index_col='timestamp')
tar     = pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Target_intra-hour.csv', parse_dates=True, index_col='timestamp')

# runs forecast for all variables and horizons
target  = ["ghi","dni"]
horizon = ["5min","10min","15min","20min","25min","30min"]
for t in target[-1:]:
    for h in horizon[-1:]:
        print("{} IH forecast for {}".format(h,t))
        target,horizon = t,h

cols = [
        "{}_{}".format(target,horizon),  # actual
        "{}_kt_{}".format(target,horizon),  # clear-sky index
        "{}_clear_{}".format(target,horizon),  # clear-sky model
        "elevation_{}".format(horizon)   # solar elevation 
    ]

train = inpEndo[inpEndo.index.year <= 2015]
train = train.join(inpExo[inpEndo.index.year <= 2015], how="inner")
train = train.join(tar[tar.index.year <= 2015], how="inner")

test = inpEndo[inpEndo.index.year == 2016]
test = test.join(inpExo[inpEndo.index.year == 2016], how="inner")
test = test.join(tar[tar.index.year == 2016], how="inner")

feature_cols = inpEndo.filter(regex=target).columns.tolist()
feature_cols_endo = inpEndo.filter(regex=target).columns.tolist()
feature_cols.extend(inpExo.columns.unique().tolist())
    
train = train[cols + feature_cols].dropna(how="any")
test  = test[cols + feature_cols].dropna(how="any")

train_X = train[feature_cols].values
test_X  = test[feature_cols].values
train_X_endo = train[feature_cols_endo].values
test_X_endo  = test[feature_cols_endo].values

train_y = train["{}_kt_{}".format(target,horizon)].values
elev_train = train["elevation_{}".format(horizon)].values
elev_test  = test["elevation_{}".format(horizon)].values

train_clear = train["{}_clear_{}".format(target,horizon)].values
test_clear = test["{}_clear_{}".format(target,horizon)].values
  

30min IH forecast for dni


# preprocessing

In [6]:
import src.utils as utils
utils.summary_stats

<function src.utils.evaluation.summary_stats(test, pred, dp, model_str, f, baseline_str='sp')>

In [7]:
from src import DataPipeline
from sklearn.preprocessing import StandardScaler

# runs forecast for all variables and horizons
target  = ["dni","ghi"]
horizon = ["5min","10min","15min","20min","25min","30min"]
for t in target[-1:]:
    for h in horizon[-1:]:
        print("{} IH forecast for {}".format(h,t))
        target,horizon = t,h
data = DataPipeline()
data.load_data()
train_test_split = data.train_test_split(target,horizon)

Xtra,Xtes,f = list(train_test_split.itterator)[0]
# normalize features
scaler = StandardScaler()
scaler.fit(Xtra)
Xtra = scaler.transform(Xtra)
Xtes = scaler.transform(Xtes)


30min IH forecast for ghi


In [8]:
baselines = pd.read_csv('../data/external/baseline_intra-day.csv', parse_dates=True, index_col='timestamp')
bl = 'sp'
baseline = baselines[f'Test_{target}_{bl}_{horizon}']

In [9]:
import numpy as np
def evaluate(labels, predictions, baseline):
    # error metrics [W/m^2]
    error = labels - predictions
    mae = np.nanmean(np.abs(error))
    mbe = np.nanmean(error)
    rmse = np.sqrt(np.nanmean(error ** 2))

    # forecast skill [-]:
    #
    #       s = 1 - RMSE_f / RMSE_p
    #
    # where RMSE_f and RMSE_p are the RMSE of the forecast model
    # and reference baseline model, respectively.
    rmse_p = np.sqrt(
        np.mean((predictions - baseline) ** 2)
    )
    skill = 1.0 - rmse / rmse_p
    return {
                "horizon": horizon,  # 5min, 10min, etc.
                "MAE": mae,
                "MBE": mbe,
                "RMSE": rmse,
                "skill": skill,
            }